In [2]:
!unzip "/content/drive/MyDrive/omniglot.zip" -d "/content/omniglot"
!unzip "/content/drive/MyDrive/runs.zip" -d "/content/"

Archive:  /content/drive/MyDrive/runs.zip
   creating: /content/runs/exp/
   creating: /content/runs/exp/checkpoint/
   creating: /content/runs/exp/tensorboard_log/
  inflating: /content/runs/exp/train_args.json  
   creating: /content/runs/exp10/
   creating: /content/runs/exp10/checkpoint/
   creating: /content/runs/exp10/tensorboard_log/
  inflating: /content/runs/exp10/tensorboard_log/events.out.tfevents.1685376194.BurtonChang.13020.0  
  inflating: /content/runs/exp10/train_args.json  
   creating: /content/runs/exp11/
   creating: /content/runs/exp11/checkpoint/
  inflating: /content/runs/exp11/checkpoint/proto_best.pth  
  inflating: /content/runs/exp11/checkpoint/proto_last.pth  
   creating: /content/runs/exp11/tensorboard_log/
  inflating: /content/runs/exp11/tensorboard_log/events.out.tfevents.1685376254.BurtonChang.15432.0  
   creating: /content/runs/exp11/tensorboard_log/Loss_TrainLoss/
  inflating: /content/runs/exp11/tensorboard_log/Loss_TrainLoss/events.out.tfevents.16

#Dataset

In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image


DATA_CACHE = {}


def read_data(csv_path):
    root = '/'.join(csv_path.split('/')[:-1])
    df = pd.read_csv(csv_path)
    df['filename'] = df['filename'].apply(lambda x: '/'.join([root, x]))
    return df


def shuffle_dim0(tensor_):
    idx = torch.randperm(tensor_.shape[0])
    return tensor_[idx]


def random_sample_dim0(tensor_, num):
    idx = torch.randperm(tensor_.shape[0])[:num]
    return tensor_[idx]


def path_to_img(img_path, img_channels, img_size):
    if img_channels == 1:
        norm = transforms.Normalize(mean=[0.5], std=[0.5])
    elif img_channels == 3:
        norm = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    else:
        raise Exception('Only 1-channel and 3-channel data are supported at present.')
    transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        norm
    ])
    img = Image.open(img_path)
    img = transform(img)
    return img


class ImageDataset(Dataset):
    def __init__(self, data_path, shot, query, img_channels, img_size):
        super(ImageDataset, self).__init__()
        self.shot = shot
        self.query = query
        self.img_channels = img_channels
        self.img_size = img_size

        df = read_data(data_path)
        self.label_imgPaths_dict = df.groupby('label').apply(lambda x: x['filename'].tolist()).to_dict()
        self.labels = list(self.label_imgPaths_dict.keys())

    def __getitem__(self, item):
        label = self.labels[item]

        if label not in DATA_CACHE:
            img_paths = self.label_imgPaths_dict[label]
            data = torch.stack([path_to_img(path, self.img_channels, self.img_size) for path in img_paths], dim=0)
            DATA_CACHE[label] = data  # shape: (num of one class samples, c, w, h)
        else:
            data = DATA_CACHE[label]

        support_data = data[:self.shot]
        query_data = data[-self.query:]
        return support_data, query_data

    def __len__(self):
        return len(self.labels)


class EpisodicBatchSampler(object):
    def __init__(self, n_classes, n_way, n_episodes):
        self.n_classes = n_classes
        self.n_way = n_way
        self.n_episodes = n_episodes

    def __len__(self):
        return self.n_episodes

    def __iter__(self):
        for i in range(self.n_episodes):
            yield torch.randperm(self.n_classes)[:self.n_way]



#utils

In [4]:
import os
import re
import glob
from pathlib import Path
import random
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import f1_score


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def set_seed(seed=1024):
    if seed < 0:
        seed = random.randint(0, 100000)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    return seed


def cosine_similarity(x, y):
    # x: N x D
    # y: M x D
    cos = nn.CosineSimilarity(dim=0)
    cos_sim = []
    for xi in x:
        cos_sim_i = []
        for yj in y:
            cos_sim_i.append(cos(xi, yj))
        cos_sim_i = torch.stack(cos_sim_i)
        cos_sim.append(cos_sim_i)
    cos_sim = torch.stack(cos_sim)
    return cos_sim  # (N, M)


def euclidean_dist_similarity(x, y):
    # x: N x D
    # y: M x D
    n = x.size(0)
    m = y.size(0)
    d = x.size(1)
    assert d == y.size(1)

    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)

    return -torch.pow(x - y, 2).sum(2)  # N*M


def metrics(pred, true):
    pred = np.array(pred).reshape(-1)
    true = np.array(true).reshape(-1)
    # acc
    acc = np.mean((pred == true))
    # f_score
    f_score = f1_score(true, pred, average='macro')
    return acc, f_score


def increment_path(path, exist_ok=False, sep='', mkdir=False):
    # Increment file or directory path, i.e. runs/exp --> runs/exp{sep}2, runs/exp{sep}3, ... etc.
    path = Path(path)  # os-agnostic
    if path.exists() and not exist_ok:
        suffix = path.suffix
        path = path.with_suffix('')
        dirs = glob.glob(f"{path}{sep}*")  # similar paths
        matches = [re.search(rf"%s{sep}(\d+)" % path.stem, d) for d in dirs]
        i = [int(m.groups()[0]) for m in matches if m]  # indices
        n = max(i) + 1 if i else 2  # increment number
        path = Path(f"{path}{sep}{n}{suffix}")  # update path
    dir = path if path.suffix == '' else path.parent  # directory
    if not dir.exists() and mkdir:
        dir.mkdir(parents=True, exist_ok=True)  # make directory
    return path

#protonet

In [7]:
import torch.nn as nn
import torch.nn.functional as F

#from utils import cosine_similarity, euclidean_dist_similarity


class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)


class ConvEncoder(nn.Module):
    def __init__(self, input_img_channels, hidden_channels):
        super(ConvEncoder, self).__init__()
        self.hidden_channels = hidden_channels
        self.input_img_channels = input_img_channels

        self.encoder = nn.Sequential(
            self.conv_block(self.input_img_channels, self.hidden_channels),
            self.conv_block(self.hidden_channels, self.hidden_channels),
            self.conv_block(self.hidden_channels, self.hidden_channels),
            self.conv_block(self.hidden_channels, self.hidden_channels),
            Flatten()
        )

    def forward(self, x):
        raw_shape = x.shape
        x = x.view(-1, *raw_shape[-3:])
        x = self.encoder(x)
        x = x.view(*raw_shape[:-3], -1)
        return x

    @staticmethod
    def conv_block(in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=7, padding=3),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )


class PrototypicalNetwork(nn.Module):
    def __init__(self, input_img_channels, hidden_channels):
        super(PrototypicalNetwork, self).__init__()
        self.encoder = ConvEncoder(input_img_channels, hidden_channels)

    def forward(self, x_support, x_query):
        """
        infer an n-way k-shot task
        :param x_support: (n, k, c, w, h)
        :param x_query: (n, q, c, w, h) or (q, c, w, h)
        :return: (q, n)
        """
        x_proto = self.encoder(x_support)  # (n, k, embed_dim)
        x_proto = x_proto.mean(1)  # (n, embed_dim)
        x_q = self.encoder(x_query)  # (n, q, embed_dim)
        x_q = x_q.view(-1, x_q.shape[-1])  # (n*q, embed_dim)

        sim_result = self.similarity(x_q, x_proto)  # (n*q, n)

        log_p_y = F.log_softmax(sim_result, dim=1)

        return log_p_y  # (n*q, n)

    @staticmethod
    def similarity(a, b, sim_type='cosine'):
        methods = {'euclidean': euclidean_dist_similarity, 'cosine': cosine_similarity}
        assert sim_type in methods.keys(), f'type must be in {methods.keys()}'
        return methods[sim_type](a, b)  # 值越大相似度越高


#train

In [8]:
#from dataset import ImageDataset, EpisodicBatchSampler
#from protonet import PrototypicalNetwork
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard.writer import SummaryWriter
import os
import json
import time
import numpy as np
from tqdm import tqdm
#from utils import set_seed, metrics, AverageMeter, increment_path


class Train_protonet(object):
    def __init__(self, args):
        self.args = args
        self.args.seed = set_seed(self.args.seed)
        self.args.result_path = increment_path(self.args.result_path)
        print(f'This training log will be saved in {str(self.args.result_path)}')

        self.args.checkpoints_dir = self.args.result_path / self.args.checkpoints_dir
        self.args.tensorboard_dir = self.args.result_path / self.args.tensorboard_dir
        if not os.path.exists(self.args.checkpoints_dir):
            os.makedirs(self.args.checkpoints_dir)
        if not os.path.exists(self.args.tensorboard_dir):
            os.makedirs(self.args.tensorboard_dir)

        # save training args
        train_args = {k: str(v) for k, v in self.args.__dict__.copy().items()}
        with open(self.args.result_path / 'train_args.json', 'w', encoding='utf-8') as f:
            json.dump(train_args, f, ensure_ascii=False, indent=4)

        self.model = self._build_model()

    def _build_model(self):
        model = PrototypicalNetwork(self.args.img_channels, self.args.hidden_channels)
        # model.cuda()
        return model

    def _get_data(self, mode='train'):
        if mode == 'train':
            data_path = self.args.train_csv_path
            way = self.args.way
            shot = self.args.shot
            query = self.args.query
            episodes = self.args.episodes
        else:
            data_path = self.args.val_csv_path
            way = self.args.val_way
            shot = self.args.val_shot
            query = self.args.val_query
            episodes = self.args.val_episodes

        data_set = ImageDataset(
            data_path=data_path,
            shot=shot,
            query=query,
            img_channels=self.args.img_channels,
            img_size=self.args.img_size,
        )
        sampler = EpisodicBatchSampler(n_classes=len(data_set), n_way=way, n_episodes=episodes)
        data_loader = DataLoader(data_set, shuffle=False, batch_sampler=sampler, num_workers=0)
        return data_loader

    def _select_optimizer(self):
        optimizer = optim.Adam(self.model.parameters(), lr=self.args.learning_rate, weight_decay=self.args.weight_decay_step)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=self.args.lr_decay_step, gamma=0.1)
        return optimizer, scheduler

    def _select_criterion(self):
        criterion = nn.NLLLoss()
        return criterion

    def eval_model(self, epoch):
        self.model.eval()
        val_losses, accuracy, f_score = [AverageMeter() for i in range(3)]
        criterion = self._select_criterion()
        with torch.no_grad():
            val_loader = self._get_data(mode='val')
            for step, (x_support, x_query) in enumerate(tqdm(val_loader, desc='Evaluating model')):
                # data (n, x, c, w, h)
                x_support, x_query = x_support, x_query
                n = x_support.shape[0]
                q = x_query.shape[1]
                y_query = torch.arange(0, n, requires_grad=False).view(n, 1).expand(n, q).reshape(n * q, )
                # infer
                output = self.model(x_support, x_query)
                # loss
                loss = criterion(output, y_query).item()
                val_losses.update(loss)
                # metrics of acc and f_score
                pred_ids = torch.argmax(output, dim=-1).cpu().numpy()
                y_query = y_query.cpu().numpy()
                acc, f_s = metrics(pred_ids, y_query)
                accuracy.update(acc)
                f_score.update(f_s)
        print(f'Epoch: {epoch} evaluation results: Val_loss: {val_losses.avg}, mAP: {accuracy.avg}, F_score: {f_score.avg}')
        return val_losses.avg, accuracy.avg, f_score.avg

    def train_one_epoch(self, optimizer, scheduler, epoch, total_epochs):
        self.model.train()
        train_losses = AverageMeter()
        criterion = self._select_criterion()
        train_loader = self._get_data(mode='train')
        epoch_iterator = tqdm(train_loader,
                              desc="Training [epoch X/X | episode X/X] (loss=X.X | lr=X.X)",
                              bar_format="{l_bar}{r_bar}",
                              dynamic_ncols=True)
        for step, (x_support, x_query) in enumerate(epoch_iterator):
            # data (n, x, c, w, h)
            x_support, x_query = x_support, x_query
            n = x_support.shape[0]
            q = x_query.shape[1]
            y_query = torch.arange(0, n, requires_grad=False).view(n, 1).expand(n, q).reshape(n * q, )

            # training one episode
            optimizer.zero_grad()
            output = self.model(x_support, x_query)
            loss = criterion(output, y_query)
            loss.backward()
            optimizer.step()

            # log
            train_losses.update(loss.item())
            epoch_iterator.set_description(
                "Training [epoch %d/%d | episode %d/%d] | (loss=%2.5f | lr=%f)" %
                (epoch, total_epochs, step + 1, len(epoch_iterator), loss.item(), scheduler.get_last_lr()[0])
            )
        scheduler.step()
        return train_losses.avg

    def train(self):
        start_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
        print(f'[{start_time}] Start Training...')
        writer = SummaryWriter(self.args.tensorboard_dir)
        best_acc = 0
        early_stop_list = []
        optimizer, scheduler = self._select_optimizer()
        for epoch in range(self.args.epochs):
            train_loss = self.train_one_epoch(optimizer, scheduler, epoch, self.args.epochs)
            val_loss, accuracy, f_score = self.eval_model(epoch)

            # logging
            writer.add_scalars('Loss', {'TrainLoss': train_loss, 'ValLoss': val_loss}, epoch)
            writer.add_scalars('Metrics', {'mAP': accuracy, 'F_score': f_score}, epoch)

            # save checkpoint
            torch.save(self.model.state_dict(), self.args.checkpoints_dir / 'proto_last.pth')
            if best_acc < accuracy:
                torch.save(self.model.state_dict(), self.args.checkpoints_dir / 'proto_best.pth')
                best_acc = accuracy

            # early stop
            early_stop_list.append(val_loss)
            if len(early_stop_list)-np.argmin(early_stop_list) > self.args.patience:
                break
        writer.close()
        end_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
        print(f'[{end_time}] End of all training. The highest accuracy is {best_acc}')
        return


#run_train

In [9]:

#from train import Train_protonet


class TrainArgs(object):
    def __init__(self):
        # record saving path
        self.result_path = './runs/exp'
        self.checkpoints_dir = 'checkpoint'
        self.tensorboard_dir = 'tensorboard_log'

        # data loader args
        self.train_csv_path = './omniglot/train.csv'
        self.val_csv_path = './omniglot/val.csv'
        self.img_channels = 1
        self.img_size = 28

        self.way = 10
        self.shot = 10
        self.query = 5
        self.episodes = 10

        self.val_way = 2
        self.val_shot = 10
        self.val_query = 5
        self.val_episodes = 10

        # train args
        self.epochs = 600
        self.patience = 50
        self.learning_rate = 0.0001
        self.lr_decay_step = 1
        self.weight_decay_step = 0
        self.seed = 42

        # model args
        self.hidden_channels = 64


if __name__ == '__main__':
    args = TrainArgs()
    exp_p = Train_protonet(args)
    exp_p.train()


This training log will be saved in runs/exp13
[2023-06-30 07:55:49] Start Training...


Training [epoch 0/600 | episode 10/10] | (loss=1.97791 | lr=0.000100): 100%|| 10/10 [00:08<00:00,  1.24it/s]
Evaluating model: 100%|██████████| 10/10 [00:00<00:00, 13.98it/s]


Epoch: 0 evaluation results: Val_loss: 0.5473827689886093, mAP: 0.9400000000000001, F_score: 0.9381313131313131


Training [epoch 1/600 | episode 10/10] | (loss=1.91748 | lr=0.000010): 100%|| 10/10 [00:07<00:00,  1.28it/s]
Evaluating model: 100%|██████████| 10/10 [00:00<00:00, 23.10it/s]


Epoch: 1 evaluation results: Val_loss: 0.5408618837594986, mAP: 0.9400000000000001, F_score: 0.9387626262626263


...

Epoch: 137 evaluation results: Val_loss: 0.5396850347518921, mAP: 0.95, F_score: 0.9494949494949495


Training [epoch 138/600 | episode 10/10] | (loss=1.95489 | lr=0.000000): 100%|| 10/10 [00:08<00:00,  1.17it/s]
Evaluating model: 100%|██████████| 10/10 [00:00<00:00, 21.66it/s]


Epoch: 138 evaluation results: Val_loss: 0.4793069511651993, mAP: 0.9800000000000001, F_score: 0.9797979797979798


Training [epoch 139/600 | episode 10/10] | (loss=1.94480 | lr=0.000000): 100%|| 10/10 [00:07<00:00,  1.34it/s]
Evaluating model: 100%|██████████| 10/10 [00:00<00:00, 15.30it/s]

Epoch: 139 evaluation results: Val_loss: 0.504892674088478, mAP: 0.9400000000000001, F_score: 0.9389646464646464
[2023-06-30 08:15:05] End of all training. The highest accuracy is 1.0
